In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv("diabetes_binary_classification_data.csv")

In [3]:
# Define features (X) and target (y)
X = data.drop('Diabetes_binary', axis=1)
y = data['Diabetes_binary']

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 15% validation, 15% test

# Create preprocessing pipelines for numeric and categorical features
numeric_features = ['BMI', 'PhysHlth', 'Age']  # Non-binary variables
categorical_features = ['GenHlth', 'HighBP', 'HighChol', 'HeartDiseaseorAttack']  # Binary variables

# Create preprocessing for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess training data (numerical and categorical features)
X_train_processed = preprocessor.fit_transform(X_train)

# Preprocess validation and test data
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

In [4]:
# Define models
log_reg = LogisticRegression(class_weight='balanced', max_iter=2000, random_state=37)
decision_tree = DecisionTreeClassifier(class_weight='balanced', max_depth=50, random_state=74)
random_forest = RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=50, random_state=20)

# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('decision_tree', decision_tree),
    ('random_forest', random_forest)],
    voting='hard')  # 'hard' for majority voting

# Fit models on the processed training data
log_reg.fit(X_train_processed, y_train)
decision_tree.fit(X_train_processed, y_train)
random_forest.fit(X_train_processed, y_train)
voting_clf.fit(X_train_processed, y_train)

# Make predictions on the validation set
log_reg_val_pred = log_reg.predict(X_val_processed)
decision_tree_val_pred = decision_tree.predict(X_val_processed)
random_forest_val_pred = random_forest.predict(X_val_processed)
voting_val_pred = voting_clf.predict(X_val_processed)

# Make predictions on the test set
log_reg_test_pred = log_reg.predict(X_test_processed)
decision_tree_test_pred = decision_tree.predict(X_test_processed)
random_forest_test_pred = random_forest.predict(X_test_processed)
voting_test_pred = voting_clf.predict(X_test_processed)

# Make predictions on the test set
log_reg_test_pred = log_reg.predict(X_test_processed)
decision_tree_test_pred = decision_tree.predict(X_test_processed)
random_forest_test_pred = random_forest.predict(X_test_processed)
voting_test_pred = voting_clf.predict(X_test_processed)

# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('decision_tree', decision_tree),
    ('random_forest', random_forest)],
    voting='hard')  # 'hard' for majority voting

In [5]:
# Function to calculate false negative rate
def calculate_fnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0  # Avoid division by zero
    return tn, fn, tp, fp, fnr

In [6]:
def print_fnr(predictions, model_name):
    accuracy = accuracy_score(y_test, predictions)
    tn, fn, tp, fp, fnr = calculate_fnr(y_test, predictions)
    
    print(f"{model_name}:")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"true Negatives: {tn}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"False Negative Rate: {fnr:.4f}")
    print(classification_report(y_test, predictions)) 

In [8]:
# Evaluate models on validation data and print results
print(f"Evaluation of models based on validation data:")
print(f"")
for model_name, predictions in zip(
    ["Logistic Regression", "Decision Tree", "Random Forest", "Voting Classifier"],
    [log_reg_val_pred, decision_tree_val_pred, random_forest_val_pred, voting_val_pred]
):
    print_fnr(predictions, model_name)

Evaluation of models based on validation data:

Logistic Regression:
Test Accuracy: 0.8372
true Negatives: 31684
False Negatives: 5021
True Positives: 173
False Positives: 1174
False Negative Rate: 0.9667
              precision    recall  f1-score   support

         0.0       0.86      0.96      0.91     32858
         1.0       0.13      0.03      0.05      5194

    accuracy                           0.84     38052
   macro avg       0.50      0.50      0.48     38052
weighted avg       0.76      0.84      0.79     38052

Decision Tree:
Test Accuracy: 0.8142
true Negatives: 30612
False Negatives: 4823
True Positives: 371
False Positives: 2246
False Negative Rate: 0.9286
              precision    recall  f1-score   support

         0.0       0.86      0.93      0.90     32858
         1.0       0.14      0.07      0.09      5194

    accuracy                           0.81     38052
   macro avg       0.50      0.50      0.50     38052
weighted avg       0.77      0.81      0.79  

In [9]:
# Evaluate models on test data and print results
for model_name, predictions in zip(
    ["Logistic Regression", "Decision Tree", "Random Forest", "Voting Classifier"],
    [log_reg_test_pred, decision_tree_test_pred, random_forest_test_pred, voting_test_pred]
):
    print_fnr(predictions, model_name)

Logistic Regression:
Test Accuracy: 0.8669
true Negatives: 32291
False Negatives: 4497
True Positives: 697
False Positives: 567
False Negative Rate: 0.8658
              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93     32858
         1.0       0.55      0.13      0.22      5194

    accuracy                           0.87     38052
   macro avg       0.71      0.56      0.57     38052
weighted avg       0.83      0.87      0.83     38052

Decision Tree:
Test Accuracy: 0.8453
true Negatives: 31171
False Negatives: 4199
True Positives: 995
False Positives: 1687
False Negative Rate: 0.8084
              precision    recall  f1-score   support

         0.0       0.88      0.95      0.91     32858
         1.0       0.37      0.19      0.25      5194

    accuracy                           0.85     38052
   macro avg       0.63      0.57      0.58     38052
weighted avg       0.81      0.85      0.82     38052

Random Forest:
Test Accuracy: 0.8498
tr